In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import pathlib

In [ ]:
csv_path = pathlib.Path("./csv/company_info.csv")

# company_info_df = pd.read_csv(csv_path, encoding='utf-8')


In [2]:
DEFAULT_CITIES = {'New York', 'Chicago', 'San Francisco', 'Austin', 'Seattle',
                  'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas',
                  'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                  'Miami'}

YOUR_CITIES = {'Boston', 'Washington DC', 'St Louis', 'San Diego',
               'San Antonio', 'Columbus', 'Sacramento', 'Charlotte', 'Memphis',
               'Detroit', 'Nashville', 'Jacksonville', 'Indianapolis',
               'Fort Worth', 'Charlotte', 'El Paso', 'Oklahoma City',
               'Las Vegas', 'Louisville', 'Milwaukee', 'Albuquerque', 'Tucson',
               'Kansas City', 'Mesa', 'Colorado Springs', 'Raleigh', 'Omaha',
               'Virginia Beach', 'Minneapolis', 'New Orleans', 'Tampa',
               'San Jose', 'Baltimore', 'Fresno', 'Oakland', 'Tulsa', 'Madison',
               'Arlington', 'Wichita', 'Cleveland', 'Aurora', 'Honolulu',
               'Orlando', 'Anchorage', 'Des Moines', 'Salt Lake City',
               'Lexington', 'Cincinnati', 'Newark', 'Durham', 'Buffalo',
               'Baton Rouge', 'Richmond', 'Boise', 'Birmingham', 'Little Rock',
               'Grand Rapids', 'Worcester', 'Providence', 'Sioux Falls',
               'Jackson', 'Hartford', 'Bridgeport', 'Jersey City', 'Charleston',
               'Billings', 'Fargo', 'Augusta'}

In [8]:
def extract_results(soup):
    return soup.find_all('div', class_='result')

In [13]:
def extract_location(result):
    """extract job location"""
    try:
        location = result.find('span', class_='location').get_text().strip()
        return location
    except:
        return None

In [14]:
def extract_company(result):
    """extract the name of the company"""
    try:
        company = result.find('span', class_='company').get_text().strip()
        return company
    except:
        return None

In [15]:
def extract_title(result):
    """extract the job title"""
    try:
        title = result.find('a', attrs={'data-tn-element': "jobTitle"}).get(
            'title')
        return title
    except:
        return None

In [16]:
def extract_salary(result):
    """extract the salary"""
    try:
        salary = result.find('td', class_='snip').\
            find('span', class_='no-wrap').\
            get_text().strip()
        return salary
    except:
        return None

In [17]:
def extract_description(result):
    """extract job description snippet"""
    try:
        description = result.find('span', class_='summary').get_text().strip()
        return description
    except:
        return None

In [18]:
def extract_review(result):
    """extract the number of reviews for the company"""
    try:
        review = result.find('a', attrs={'data-tn-element': "reviewStars"})
        review = review.find('span', class_="slNoUnderline")
        review = review.get_text().strip()
        # extract only the number
        review = review.replace(',', '').replace(' reviews', '')
        return review
    except:
        return None

In [54]:
def extract_star(result):
    """extract a number (width) that is proportional to the number of stars
    shown for the company"""
    try:
        # the 'style' attribute dictates how many stars are filled with color
        star = result.find('span', class_='ratingsContent').get_text()
        # extract only the number
        star = star.replace('""', '').replace('\n', '')
        return star
    except:
        return None


In [36]:
url = "http://www.indeed.com/jobs"
params = {'q': 'data scientist', 'radius': '100'}
# params = {'radius': '100'}
max_results = 1

In [ ]:
# for city in DEFAULT_CITIES | YOUR_CITIES:
for city in DEFAULT_CITIES:
    for start in range(0, max_results, 10):
        url_params = params.copy()
        url_params.update({'l': city, 'start': start})
        scraped_data = {'location': [],
                    'company': [],
                    'title': [],
#                     'salary': [],
#                     'description': [],
#                     'review': [],
                    'star': []}
        response = requests.get(url, params=url_params)
        soup = BeautifulSoup(response.text, 'lxml')
        results = soup.find_all('div', class_='result')
        
        time.sleep(0.5)
        print(response.url)
        print('------------------------------------------------------------------------')
        
        for result in results:
            scraped_data['location'].append(extract_location(result))
            scraped_data['company'].append(extract_company(result))
            scraped_data['title'].append(extract_title(result))
#             scraped_data['salary'].append(extract_salary(result))
#             scraped_data['description'].append(extract_description(result))
#             scraped_data['review'].append(extract_review(result))
            scraped_data['star'].append(extract_star(result))
        company_info_df = pd.DataFrame(scraped_data)

In [56]:
company_info_df

,location,company,title,salary,description,review,star
0,None,TTEC,Customer Service Representative - Work from Home,None,None,None,3.5
1,None,"Commercial Beverage Concepts, LLC",Warehouse Financial Analyst,None,None,None,None
2,None,GEORGIA-PACIFIC,PLYWOOD TRAINEE,None,None,None,3.4
3,"Houston, TX",HEB,Curbside In Store Shopper,None,None,None,4.3
4,"Houston, TX",HEB,Checker,None,None,None,4.3
5,"Houston, TX",HEB,FT Curbside Shopper,None,None,None,4.3
6,"Rosenberg, TX",Frito-Lay North America,Warehouse/Material Handler,None,None,None,3.4
7,"Houston, TX 77090",NINJA Nation - Aflac,Insurance Agent - Training Provided & Flexible...,None,None,None,None
8,"Sugar Land, TX",HEB,Night Stocker,None,None,None,4.3
9,"Houston, TX 77002 (Downtown area)",Harris County,Harris County Sheriff's Office Career Opportun...,None,None,None,3.7
